In [1]:
from CTGAN_1_7 import *

In [ ]:
import pandas as pd

real_data = pd.read_csv('../Data/Train Test/train_data.csv')[['Promoter Sequence', 'Normalized Expression']]
real_data.head()

In [ ]:
real_data = expand_sequences(real_data)
real_data.head()

In [ ]:
from sdv.metadata import Metadata

# 1. auto-detect metadata based in your data
metadata = Metadata.detect_from_dataframes(
    data={
        'sequences': real_data,
    })

# 2. carefully inspect and update your metadata
metadata.visualize()
for column in real_data.columns[:-1]:
    metadata.update_column(
        column_name=column,
        sdtype='categorical',
        table_name='sequences'
    )
metadata.update_column(
    column_name='Normalized Expression',
    sdtype='numerical',
    table_name='sequences'
)

In [ ]:
from sdv.single_table import CTGANSynthesizer

synthesizer = CTGANSynthesizer(metadata, epochs=100, verbose=True)
synthesizer.fit(real_data)

In [ ]:
synthetic_data = synthesizer.sample(num_rows=500)
synthetic_data.head()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic

diagnostic = run_diagnostic(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata
)